In [1]:
from catboost import CatBoostRegressor, Pool
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
class MultiQuantileRegressor:
    def __init__(self, n_quantiles = 10, #number of quantiles to be used
                 hyperparams:dict = {}, #model hyperparameters
                 rs:int = 1, #random seed
                 categorical_features:list = [], 
                 numerical_features:list = [], 
                 ):
        
        self.model = None
        #cuantos cuantiles queremos
        self.n_quantiles = n_quantiles
        self.quantiles = [q/n_quantiles for q in range(1, n_quantiles)]
        #Se necesita definir este string para llamar a la funcion de perdida del modelo
        self.quantile_str = str(self.quantiles).replace('[','').replace(']','').replace(' ', '')

        hyperparams['loss_function'] = f'MultiQuantile:alpha={self.quantile_str}'
        hyperparams['random_seed'] = rs

        self.hyperparams = hyperparams
        self.predictive_features = categorical_features + numerical_features
        self.categorical_features = categorical_features
        self.numerical_features = numerical_features
        
    def instantiate_model(self):
        self.model = CatBoostRegressor(**self.hyperparams)

    def fit(self,
            xtrain:pd.DataFrame, 
            ytrain:pd.Series,
            xval:pd.DataFrame = None,
            yval:pd.Series = None,
            verbose = 0):
        
        N_train = len(xtrain)
        train_pool = Pool(xtrain, ytrain, cat_features=self.categorical_features)
                
        self.instantiate_model()

        metrics = dict()

        if (xval is not None) & (yval is not None):
            N_val = len(xval)
            val_pool = Pool(xval, yval, cat_features=self.categorical_features) 
            self.model.fit(train_pool, 
                           eval_set=val_pool, 
                           verbose=verbose, 
                           plot=False, 
                           use_best_model=True)
            
            #train metrics
            preds = self.model.predict(xtrain)
            point_preds = preds.mean(axis=1)
            mape = mean_absolute_percentage_error(ytrain, point_preds)
            mean_spread = np.mean(np.abs(preds[:,-1] - preds[:,0]))
            standard_deviation = np.std(preds, axis=1).mean()
            loss_fn = self.model.get_best_score()['learn'][f'MultiQuantile:alpha={self.quantile_str}']
            train_preds = pd.DataFrame(preds, columns=[f'pred_{q}' for q in self.quantiles])

            metrics['training'] = dict()
            metrics['training']['mape'] = mape
            metrics['training']['mean_spread'] = mean_spread
            metrics['training']['standard_deviation'] = standard_deviation
            metrics['training']['loss_fn'] = loss_fn
            
            #Validation performance
            preds = self.model.predict(xval)
            point_preds = preds.mean(axis=1)

            mape = mean_absolute_percentage_error(yval, point_preds)
            mean_spread = np.mean(preds[:,-1] - preds[:,0])
            standard_deviation = np.std(preds, axis=1).mean()
            loss_fn = self.model.get_best_score()['validation'][f'MultiQuantile:alpha={self.quantile_str}']
            val_preds = pd.DataFrame(preds, columns=[f'pred_{q}' for q in self.quantiles])

            metrics['validation'] = dict()
            metrics['validation']['mape'] = mape
            metrics['validation']['mean_spread'] = mean_spread
            metrics['validation']['standard_deviation'] = standard_deviation
            metrics['validation']['loss_fn'] = loss_fn

            metadata = dict()
            metadata['quantiles'] = self.quantiles
            metadata['quantile_str'] = self.quantile_str
            metadata['hyperparams'] = self.hyperparams
            metadata['full_hyperparams'] = self.model.get_all_params()

            metadata['categorical_features'] = self.categorical_features
            metadata['numerical_features'] = self.numerical_features
            metadata['train_size'] = N_train
            metadata['val_size'] = N_val

            predictions = dict()
            predictions['train'] = train_preds
            predictions['val'] = val_preds

            return predictions, metrics, metadata
            

        else:
            self.model.fit(train_pool, 
                           verbose=verbose, 
                           plot=False, 
                           use_best_model=True)
            
            #train metrics
            preds = self.model.predict(xtrain)
            point_preds = preds.mean(axis=1)
            mape = mean_absolute_percentage_error(ytrain, point_preds)
            mean_spread = np.mean(preds[:,-1] - preds[:,0])
            standard_deviation = np.std(preds, axis=1).mean()
            loss_fn = self.model.get_best_score()['learn'][f'MultiQuantile:alpha={self.quantile_str}']
            train_preds = pd.DataFrame(preds, columns=[f'pred_{q}' for q in self.quantiles])

            metrics['training'] = dict()
            metrics['training']['mape'] = mape
            metrics['training']['mean_spread'] = mean_spread
            metrics['training']['standard_deviation'] = standard_deviation
            metrics['training']['loss_fn'] = loss_fn

            metadata = dict()
            metadata['quantiles'] = self.quantiles
            metadata['quantile_str'] = self.quantile_str
            metadata['hyperparams'] = self.hyperparams
            metadata['full_hyperparams'] = self.model.get_all_params()
            metadata['categorical_features'] = self.categorical_features
            metadata['numerical_features'] = self.numerical_features
            metadata['train_size'] = N_train

            predictions = dict()
            predictions['train'] = train_preds

            return predictions, metrics, metadata
        
    def predict(self, xtest:pd.DataFrame, return_point_preds = True, return_preds_as_df = False):
        preds = self.model.predict(xtest)
        point_preds = preds.mean(axis=1)            

        if return_preds_as_df:
            preds = pd.DataFrame(preds, columns=[f'pred_{q}' for q in self.quantiles])

        if return_point_preds:
            return preds, point_preds
        
        else:
            return preds
    
    def evaluate(self, ytrue:pd.Series, point_preds, preds):
        mape = mean_absolute_percentage_error(ytrue, point_preds)
        mean_spread = np.mean(preds[:,-1] - preds[:,0])
        standard_deviation = np.std(preds, axis=1).mean()

        metrics = dict()
        metrics['mape'] = mape
        metrics['mean_spread'] = mean_spread
        metrics['standard_deviation'] = standard_deviation
        return metrics

Ejemplo de uso

In [3]:
#Dummy data (este dataset se tarda un poco en cargar)
from catboost.datasets import monotonic1

monotonic1_train, monotonic1_test = monotonic1()

monotonic1_train

,Target,Num0,Cat0,Cat1,Cat2,Cat3,Cat4,Cat5,Num1,MonotonicNeg0,...,MonotonicNeg8,Num22,MonotonicNeg9,Num23,MonotonicNeg10,Num24,Num25,Num26,Num27,Num28
0,1,0.078947,6780,7,66,494,583,109,0.020319,0.066393,...,0.059776,0.089834,0.019978,0.057514,0.019833,0.220801,0.088723,0.316292,0.999997,0.999994
1,1,0.039474,41183,8,58,472,650,109,0.000608,0.078344,...,0.010702,0.004125,0.011961,0.057514,0.000727,0.847325,0.999976,0.811932,0.017582,0.999994
2,0,0.144737,52666,5,32,403,665,109,0.017192,0.074828,...,0.343446,0.019208,0.084036,0.057514,0.018917,0.996521,0.249741,0.175907,0.004534,0.999994
3,0,0.000000,2451,8,25,514,583,109,0.000863,0.066093,...,0.014197,0.089834,0.000866,0.057514,0.000859,0.847325,0.077754,0.086481,0.999997,0.999994
4,0,0.105263,27005,9,50,189,219,109,0.003540,0.066301,...,0.015038,0.046838,0.006607,0.057514,0.003469,0.296760,0.046695,0.131224,0.008043,0.999994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827042,0,0.092105,11473,2,42,309,583,109,0.005911,0.066346,...,0.016001,0.089834,0.005824,0.057514,0.005781,0.174054,0.055488,0.006744,0.999997,0.999994
2827043,0,0.131579,10343,8,58,61,163,109,0.009518,0.075443,...,0.086576,0.024275,0.037737,0.057514,0.010570,0.847325,0.999976,0.953237,0.014712,0.999994
2827044,0,0.118421,33709,7,89,480,583,109,0.002931,0.063947,...,0.008424,0.089834,0.002794,0.057514,0.002774,0.220801,0.091011,0.204470,0.999997,0.999994
2827045,0,0.000000,51422,5,4,218,332,109,0.002770,0.064556,...,0.007381,0.017842,0.012611,0.057514,0.002647,0.996521,0.307104,0.202947,0.006410,0.999994


In [4]:
monotonic1_test

,Target,Num0,Cat0,Cat1,Cat2,Cat3,Cat4,Cat5,Num1,MonotonicNeg0,...,MonotonicNeg8,Num22,MonotonicNeg9,Num23,MonotonicNeg10,Num24,Num25,Num26,Num27,Num28
0,0,0.144737,21599,12,80,390,583,109,0.001302,0.084337,...,0.008192,0.089834,0.001356,0.057514,0.001346,0.154136,0.108807,0.214663,0.999997,0.999994
1,0,0.013158,29748,0,47,171,681,51,0.008405,0.083899,...,0.045563,0.185618,0.004185,0.054323,0.008992,0.312872,0.192753,0.170700,0.001924,0.000186
2,0,0.144737,10423,7,86,113,583,109,0.004338,0.080834,...,0.009681,0.089834,0.004277,0.057514,0.004246,0.220801,0.052217,0.023535,0.999997,0.999994
3,0,0.092105,48892,9,77,284,583,109,0.038159,0.082278,...,0.018629,0.089834,0.038117,0.057514,0.037840,0.296760,0.251499,0.131705,0.999997,0.999994
4,0,0.000000,5832,0,47,427,176,58,0.017210,0.078808,...,0.019977,0.142491,0.010450,0.135979,0.007114,0.312872,0.192753,0.210305,0.008542,0.001604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706756,0,0.131579,32633,1,125,425,271,109,0.000868,0.074214,...,0.012292,0.008261,0.007550,0.057514,0.000796,0.224415,0.163928,0.362164,0.029672,0.999994
706757,1,0.039474,1095,8,58,345,371,109,0.009101,0.085608,...,0.120729,0.022934,0.035432,0.057514,0.009408,0.847325,0.999976,0.996935,0.003780,0.999994
706758,0,0.052632,14327,0,28,219,67,109,0.033079,0.083003,...,0.028889,0.233300,0.012965,0.057514,0.033095,0.312872,0.029615,0.060711,0.008261,0.999994
706759,1,0.026316,28225,1,125,425,496,112,0.001540,0.086557,...,0.025086,0.015957,0.008614,0.005506,0.011725,0.224415,0.163928,0.362164,0.021229,0.000230


In [5]:
monotonic1_train_sample = monotonic1_train.sample(frac=0.1, random_state=42)
monotonic1_test_sample = monotonic1_test.sample(frac=0.1, random_state=42)

In [6]:
ytrain = monotonic1_train_sample['Num26']
xtrain = monotonic1_train_sample.drop('Num26', axis=1)
yval = monotonic1_test_sample['Num26']
xval = monotonic1_test_sample.drop('Num26', axis=1)

categorical_features = [_ for _ in xtrain.columns if 'Cat' in _]
numerical_features = [_ for _ in xtrain.columns if 'Cat' not in _]

### Entrenamiento:

In [7]:
regressor = MultiQuantileRegressor(n_quantiles=10, 
                                   hyperparams={'iterations': 250}, #En la práctica usaríamos entre 750-2000 iteraciones/estimadores
                                   rs = 123,
                                   categorical_features=categorical_features, #No necesitamos preprocesarlas en catboost, solo indicarlas
                                   numerical_features=numerical_features)

In [8]:
predictions, metrics, metadata = regressor.fit(xtrain, ytrain, xval, yval, verbose=50)

0:	learn: 0.0882395	test: 0.0878919	best: 0.0878919 (0)	total: 572ms	remaining: 2m 22s
50:	learn: 0.0242622	test: 0.0241722	best: 0.0241722 (50)	total: 21s	remaining: 1m 22s
100:	learn: 0.0093594	test: 0.0093459	best: 0.0093459 (100)	total: 41.1s	remaining: 1m
150:	learn: 0.0059229	test: 0.0059240	best: 0.0059240 (150)	total: 1m 1s	remaining: 40.6s
200:	learn: 0.0048773	test: 0.0048808	best: 0.0048808 (200)	total: 1m 23s	remaining: 20.4s
249:	learn: 0.0045001	test: 0.0045036	best: 0.0045036 (249)	total: 1m 45s	remaining: 0us

bestTest = 0.004503581401
bestIteration = 249



In [9]:
predictions['train']

,pred_0.1,pred_0.2,pred_0.3,pred_0.4,pred_0.5,pred_0.6,pred_0.7,pred_0.8,pred_0.9
0,0.952450,0.952522,0.952676,0.952837,0.953052,0.953247,0.953251,0.953353,0.953563
1,0.018539,0.023862,0.028049,0.032283,0.036063,0.040002,0.045450,0.052822,0.068108
2,0.810938,0.811246,0.811127,0.810999,0.811164,0.811715,0.811803,0.812020,0.812346
3,0.366907,0.366903,0.367679,0.367209,0.368257,0.368597,0.367057,0.366053,0.366186
4,0.120830,0.125480,0.128630,0.130801,0.134454,0.141840,0.146606,0.150841,0.156097
...,...,...,...,...,...,...,...,...,...
282700,0.015281,0.019551,0.021556,0.022117,0.024689,0.027473,0.027246,0.030640,0.035859
282701,0.306689,0.309225,0.312171,0.312760,0.314339,0.314840,0.319822,0.324156,0.338221
282702,0.193409,0.198554,0.202074,0.213860,0.217458,0.225361,0.248109,0.263456,0.330609
282703,0.804705,0.808478,0.814280,0.812195,0.816745,0.817979,0.812250,0.811387,0.811431


In [10]:
metrics['training'], metrics['validation']

({'mape': 731590260.3628671,
  'mean_spread': 0.04437777083561119,
  'standard_deviation': 0.01357121850753662,
  'loss_fn': 0.004500068860674632},
 {'mape': 0.4268282301324402,
  'mean_spread': 0.04483031199867458,
  'standard_deviation': 0.013718164062011156,
  'loss_fn': 0.004503581400626787})

In [11]:
monotonic1_test_sample2 = monotonic1_test.sample(frac=0.1, random_state=1)
ytest = monotonic1_test_sample['Num26']
xtest = monotonic1_test_sample.drop('Num26', axis=1)

### Evaluacion de un conjunto de datos:

In [12]:
preds, point_preds = regressor.predict(xtest)

In [13]:
preds

array([[0.1778116 , 0.19446191, 0.20890857, ..., 0.2334156 , 0.23840805,
        0.26457867],
       [0.61730667, 0.61523327, 0.62484295, ..., 0.71335756, 0.72919267,
        0.74927218],
       [0.35906382, 0.36120366, 0.36240401, ..., 0.36816183, 0.37018748,
        0.36911051],
       ...,
       [0.80470473, 0.80847841, 0.81428019, ..., 0.81224959, 0.81138668,
        0.81143129],
       [0.10778458, 0.11543631, 0.11931929, ..., 0.12657087, 0.12786601,
        0.13798772],
       [0.0023956 , 0.0043332 , 0.00648543, ..., 0.01570386, 0.02036936,
        0.03048355]])

In [14]:
point_preds

array([0.22253445, 0.66907684, 0.36560461, ..., 0.8121611 , 0.12231927,
       0.01234167])

In [15]:
regressor.evaluate(ytest, point_preds, preds)

{'mape': 0.4268282301324402,
 'mean_spread': 0.04483031199867458,
 'standard_deviation': 0.013718164062011156}

In [16]:
#podemos regresar las predicciones como dataframe para otros análisis
preds, point_preds = regressor.predict(xtest, return_preds_as_df=True)

In [17]:
preds

,pred_0.1,pred_0.2,pred_0.3,pred_0.4,pred_0.5,pred_0.6,pred_0.7,pred_0.8,pred_0.9
0,0.177812,0.194462,0.208909,0.225354,0.230197,0.229675,0.233416,0.238408,0.264579
1,0.617307,0.615233,0.624843,0.638184,0.660316,0.673986,0.713358,0.729193,0.749272
2,0.359064,0.361204,0.362404,0.365241,0.367757,0.367312,0.368162,0.370187,0.369111
3,0.996288,0.996320,0.996378,0.996465,0.996516,0.996536,0.996600,0.996710,0.996921
4,0.043839,0.050687,0.063207,0.068423,0.070597,0.071955,0.074997,0.076585,0.078932
...,...,...,...,...,...,...,...,...,...
70671,0.005218,0.006261,0.009067,0.011995,0.012752,0.013784,0.015958,0.020534,0.026371
70672,0.200989,0.200992,0.201048,0.200917,0.201159,0.201257,0.201373,0.201784,0.204127
70673,0.804705,0.808478,0.814280,0.812195,0.816745,0.817979,0.812250,0.811387,0.811431
70674,0.107785,0.115436,0.119319,0.120728,0.122612,0.122568,0.126571,0.127866,0.137988


In [18]:
#Verificar si el valor verdadero está dentro del 'intervalo de confianza' (predicción)
preds['Num26'] = ytest.values
preds['in_interval'] = preds.apply(lambda x: x['Num26'] >= x['pred_0.1'] and x['Num26'] <= x['pred_0.9'], axis=1)

In [20]:
preds['in_interval'].value_counts(normalize=True)

in_interval
True     0.833833
False    0.166167
Name: proportion, dtype: float64